In [1]:
#!/usr/bin/env python

# --------------------------------------------------------
# Py Faster R-CNN train notebook 
# based on py-faster-rcnn Written by Ross Girshick from Microsoft (MIT License)
# Edited by Joris GUERRY from ONERA
# --------------------------------------------------------

"""Train a Faster R-CNN network using alternating optimization.
This tool implements the alternating optimization algorithm described in our
NIPS 2015 paper ("Faster R-CNN: Towards Real-time Object Detection with Region
Proposal Networks." Shaoqing Ren, Kaiming He, Ross Girshick, Jian Sun.)
"""

import _init_paths
from fast_rcnn.train import get_training_roidb, train_net
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from datasets.factory import get_imdb
from rpn.generate import imdb_proposals
import argparse
import pprint
import numpy as np
import sys, os
import multiprocessing as mp
import cPickle
import shutil

In [2]:
def get_roidb(imdb_name, rpn_file=None):
    print 'Requiring dataset `{:s}` for training'.format(imdb_name)
    imdb = get_imdb(imdb_name)
    print 'Loaded dataset `{:s}` for training'.format(imdb.name)
    imdb.set_proposal_method(cfg.TRAIN.PROPOSAL_METHOD)
    print 'Set proposal method: {:s}'.format(cfg.TRAIN.PROPOSAL_METHOD)
    if rpn_file is not None:
        imdb.config['rpn_file'] = rpn_file
    roidb = get_training_roidb(imdb)
    return roidb, imdb

In [3]:
def get_solvers(net_name):
    # Faster R-CNN Alternating Optimization
    n = 'faster_rcnn_alt_opt'
    # Solver for each training stage
    solvers = [[net_name, n, 'stage1_rpn_solver60k80k.pt'],
               [net_name, n, 'stage1_fast_rcnn_solver30k40k.pt'],
               [net_name, n, 'stage2_rpn_solver60k80k.pt'],
               [net_name, n, 'stage2_fast_rcnn_solver30k40k.pt']]
    solvers = [os.path.join(cfg.MODELS_DIR, *s) for s in solvers]
    # Iterations for each training stage
    #max_iters = [80000, 40000, 80000, 40000]
    max_iters = [100, 100, 100, 100]
    # Test prototxt for the RPN
    rpn_test_prototxt = os.path.join(
        cfg.MODELS_DIR, net_name, n, 'rpn_test.pt')
    return solvers, max_iters, rpn_test_prototxt

In [4]:
def _init_caffe(cfg):
    """Initialize pycaffe in a training process.
    """

    import caffe
    # fix the random seeds (numpy and caffe) for reproducibility
    np.random.seed(cfg.RNG_SEED)
    caffe.set_random_seed(cfg.RNG_SEED)
    # set up caffe
    caffe.set_mode_gpu()
    caffe.set_device(cfg.GPU_ID)

In [5]:
class Args:
    def __init__(self,
                 gpu_id=0,
                 net_name='VGG_CNN_M_1024',
                 pretrained_model='/data/workspace/jg_pfr/data/imagenet_models/VGG_CNN_M_1024.v2.caffemodel',
                 cfg_file='/data/workspace/jg_pfr/experiments/cfgs/faster_rcnn_alt_opt.yml',
                 imdb_name= 'voc_2007_fake',
                 set_cfgs=None):
        
        self.gpu_id = gpu_id
        self.net_name = net_name
        self.pretrained_model = pretrained_model
        self.cfg_file = cfg_file
        self.imdb_name = imdb_name
        self.set_cfgs = set_cfgs
    
args = Args()

if args.cfg_file is not None:
    cfg_from_file(args.cfg_file)
if args.set_cfgs is not None:
    cfg_from_list(args.set_cfgs)
cfg.GPU_ID = args.gpu_id

#pprint.pprint(cfg)

In [6]:
# solves, iters, etc. for each training stage
solvers, max_iters, rpn_test_prototxt = get_solvers(args.net_name)

In [7]:
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
print 'Stage 1.1 RPN, init from ImageNet model'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

"""Train a Region Proposal Network in a separate training process.
"""

# Not using any proposals, just ground-truth boxes
cfg.TRAIN.HAS_RPN = True
cfg.TRAIN.BBOX_REG = False  # applies only to Fast R-CNN bbox regression
cfg.TRAIN.PROPOSAL_METHOD = 'gt'
cfg.TRAIN.IMS_PER_BATCH = 1
print 'Init model: {}'.format(args.pretrained_model)
print('Using config:')
pprint.pprint(cfg)

import caffe
_init_caffe(cfg)

roidb, imdb = get_roidb(args.imdb_name)
print 'roidb len: {}'.format(len(roidb))
output_dir = get_output_dir(imdb)
print 'Output will be saved to `{:s}`'.format(output_dir)

model_paths = train_net(solvers[0], roidb, output_dir,
                        pretrained_model=args.pretrained_model,
                        max_iters=max_iters[0])

# Cleanup all but the final model
for i in model_paths[:-1]:
    os.remove(i)
rpn_model_path = str(model_paths[-1])

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Stage 1.1 RPN, init from ImageNet model
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Init model: /data/workspace/jg_pfr/data/imagenet_models/VGG_CNN_M_1024.v2.caffemodel
Using config:
{'DATA_DIR': '/data/workspace/jg_pfr/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'faster_rcnn_alt_opt',
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MODELS_DIR': '/data/workspace/jg_pfr/models/pascal_voc',
 'PIXEL_MEANS': array([[[ 102.9801,  115.9465,  122.7717]]]),
 'RNG_SEED': 3,
 'ROOT_DIR': '/data/workspace/jg_pfr',
 'TEST': {'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'selective_search',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 300,
          'RPN_PRE_NMS_TOP_N': 6000,
          'SCALES': [600],
          'SVM': False},
 'TRAIN': {'ASPECT_GROUPING': True,
           'BATCH_SIZE': 128,
     

In [8]:
#rpn_model_path = '/data/workspace/jg_pfr/output/faster_rcnn_alt_opt/voc_2007_val/vgg_cnn_m_1024_rpn_iter_100.caffemodel'

In [9]:
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
print 'Stage 1.2 RPN, generate proposals'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

cfg.TEST.RPN_PRE_NMS_TOP_N = -1     # no pre NMS filtering
cfg.TEST.RPN_POST_NMS_TOP_N = 2000  # limit top boxes after NMS
print 'RPN model: {}'.format(rpn_model_path)
print('Using config:')
pprint.pprint(cfg)

import caffe
_init_caffe(cfg)

# NOTE: the matlab implementation computes proposals on flipped images, too.
# We compute them on the image once and then flip the already computed
# proposals. This might cause a minor loss in mAP (less proposal jittering).
imdb = get_imdb(args.imdb_name)
print 'Loaded dataset `{:s}` for proposal generation'.format(imdb.name)

# Load RPN and configure output directory
rpn_net = caffe.Net(rpn_test_prototxt, rpn_model_path, caffe.TEST)
output_dir = get_output_dir(imdb)
print 'Output will be saved to `{:s}`'.format(output_dir)

# Generate proposals on the imdb
rpn_proposals = imdb_proposals(rpn_net, imdb)

# Write proposals to disk
rpn_net_name = os.path.splitext(os.path.basename(rpn_model_path))[0]
rpn_proposals_path = os.path.join(
    output_dir, rpn_net_name + '_proposals.pkl')
with open(rpn_proposals_path, 'wb') as f:
    cPickle.dump(rpn_proposals, f, cPickle.HIGHEST_PROTOCOL)
print 'Wrote RPN proposals to {}'.format(rpn_proposals_path)
print "Number of images : "+ str(len(rpn_proposals))
print "Number of BB proposal : "+ str((rpn_proposals[0]).shape[0])

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Stage 1.2 RPN, generate proposals
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
RPN model: /data/workspace/jg_pfr/output/faster_rcnn_alt_opt/voc_2007_fake/vgg_cnn_m_1024_rpn_iter_100.caffemodel
Using config:
{'DATA_DIR': '/data/workspace/jg_pfr/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'faster_rcnn_alt_opt',
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MODELS_DIR': '/data/workspace/jg_pfr/models/pascal_voc',
 'PIXEL_MEANS': array([[[ 102.9801,  115.9465,  122.7717]]]),
 'RNG_SEED': 3,
 'ROOT_DIR': '/data/workspace/jg_pfr',
 'TEST': {'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'selective_search',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 2000,
          'RPN_PRE_NMS_TOP_N': -1,
          'SCALES': [600],
          'SVM': False},
 'TRAIN': {'ASPECT_GROUPING': True,
           'B

In [10]:
#rpn_proposals_path = '/data/workspace/jg_pfr/output/faster_rcnn_alt_opt/voc_2007_val/vgg_cnn_m_1024_rpn_iter_100_proposals.pkl'

In [11]:
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
print 'Stage 1.3 Fast R-CNN using RPN proposals, init from ImageNet model'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

cfg.TRAIN.SNAPSHOT_INFIX = 'stage1'
cfg.TRAIN.HAS_RPN = False           # not generating prosals on-the-fly
cfg.TRAIN.PROPOSAL_METHOD = 'rpn'   # use pre-computed RPN proposals instead
cfg.TRAIN.IMS_PER_BATCH = 2
print 'Init model: {}'.format(args.pretrained_model)
print 'RPN proposals: {}'.format(rpn_proposals_path)
print('Using config:')
pprint.pprint(cfg)

import caffe
_init_caffe(cfg)

roidb, imdb = get_roidb(args.imdb_name, rpn_file=rpn_proposals_path)
output_dir = get_output_dir(imdb)
print 'Output will be saved to `{:s}`'.format(output_dir)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Stage 1.3 Fast R-CNN using RPN proposals, init from ImageNet model
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Init model: /data/workspace/jg_pfr/data/imagenet_models/VGG_CNN_M_1024.v2.caffemodel
RPN proposals: /data/workspace/jg_pfr/output/faster_rcnn_alt_opt/voc_2007_fake/vgg_cnn_m_1024_rpn_iter_100_proposals.pkl
Using config:
{'DATA_DIR': '/data/workspace/jg_pfr/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'faster_rcnn_alt_opt',
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MODELS_DIR': '/data/workspace/jg_pfr/models/pascal_voc',
 'PIXEL_MEANS': array([[[ 102.9801,  115.9465,  122.7717]]]),
 'RNG_SEED': 3,
 'ROOT_DIR': '/data/workspace/jg_pfr',
 'TEST': {'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'selective_search',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 2000,
          '

In [12]:
print solvers[1]
print output_dir
print args.pretrained_model
print max_iters[1]

/data/workspace/jg_pfr/models/pascal_voc/VGG_CNN_M_1024/faster_rcnn_alt_opt/stage1_fast_rcnn_solver30k40k.pt
/data/workspace/jg_pfr/output/faster_rcnn_alt_opt/voc_2007_fake
/data/workspace/jg_pfr/data/imagenet_models/VGG_CNN_M_1024.v2.caffemodel
100


In [13]:
pprint.pprint(cfg)

{'DATA_DIR': '/data/workspace/jg_pfr/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'faster_rcnn_alt_opt',
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MODELS_DIR': '/data/workspace/jg_pfr/models/pascal_voc',
 'PIXEL_MEANS': array([[[ 102.9801,  115.9465,  122.7717]]]),
 'RNG_SEED': 3,
 'ROOT_DIR': '/data/workspace/jg_pfr',
 'TEST': {'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'selective_search',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 2000,
          'RPN_PRE_NMS_TOP_N': -1,
          'SCALES': [600],
          'SVM': False},
 'TRAIN': {'ASPECT_GROUPING': True,
           'BATCH_SIZE': 128,
           'BBOX_INSIDE_WEIGHTS': [1.0, 1.0, 1.0, 1.0],
           'BBOX_NORMALIZE_MEANS': [0.0, 0.0, 0.0, 0.0],
           'BBOX_NORMALIZE_STDS': [0.1, 0.1, 0.2, 0.2],
           'BBOX_NORMALIZE_TARGETS': True,
           'BBOX_NORMALIZE_TARGETS_PRECOMPUTED': Fa

In [14]:
# Train Fast R-CNN
model_paths = train_net(solvers[1], roidb, output_dir,
                        pretrained_model=args.pretrained_model,
                        max_iters=max_iters[1])
# Cleanup all but the final model
for i in model_paths[:-1]:
    os.remove(i)
fast_rcnn_model_path = model_paths[-1]
# Send Fast R-CNN model path over the multiprocessing queue
#queue.put({'model_path': fast_rcnn_model_path})

Filtered 0 roidb entries: 200 -> 200
RoiDataLayer: name_to_top: {'rois': 1, 'labels': 2, 'data': 0}


AssertionError: 

In [ ]:
stoooooooooooooooooooooooooooop here Jack !

In [ ]:
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
print 'Stage 2 RPN, init from stage 1 Fast R-CNN model'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

cfg.TRAIN.SNAPSHOT_INFIX = 'stage2'
mp_kwargs = dict(
        queue=mp_queue,
        imdb_name=args.imdb_name,
        init_model=str(fast_rcnn_stage1_out['model_path']),
        solver=solvers[2],
        max_iters=max_iters[2],
        cfg=cfg)
p = mp.Process(target=train_rpn, kwargs=mp_kwargs)
p.start()
rpn_stage2_out = mp_queue.get()
p.join()

In [ ]:
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
print 'Stage 2 RPN, generate proposals'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

mp_kwargs = dict(
        queue=mp_queue,
        imdb_name=args.imdb_name,
        rpn_model_path=str(rpn_stage2_out['model_path']),
        cfg=cfg,
        rpn_test_prototxt=rpn_test_prototxt)
p = mp.Process(target=rpn_generate, kwargs=mp_kwargs)
p.start()
rpn_stage2_out['proposal_path'] = mp_queue.get()['proposal_path']
p.join()

In [ ]:
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
print 'Stage 2 Fast R-CNN, init from stage 2 RPN R-CNN model'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

cfg.TRAIN.SNAPSHOT_INFIX = 'stage2'
mp_kwargs = dict(
        queue=mp_queue,
        imdb_name=args.imdb_name,
        init_model=str(rpn_stage2_out['model_path']),
        solver=solvers[3],
        max_iters=max_iters[3],
        cfg=cfg,
        rpn_file=rpn_stage2_out['proposal_path'])
p = mp.Process(target=train_fast_rcnn, kwargs=mp_kwargs)
p.start()
fast_rcnn_stage2_out = mp_queue.get()
p.join()

In [ ]:
# Create final model (just a copy of the last stage)
final_path = os.path.join(
        os.path.dirname(fast_rcnn_stage2_out['model_path']),
        args.net_name + '_faster_rcnn_final.caffemodel')
print 'cp {} -> {}'.format(
        fast_rcnn_stage2_out['model_path'], final_path)
shutil.copy(fast_rcnn_stage2_out['model_path'], final_path)
print 'Final model: {}'.format(final_path)